# Fine-tuning of the pre-trained seBERT model for the sentiment classification task
This notebook can be used to fine-tune a pre-trained seBERT model for any sequence classification task.
We use the example of sentiment mining which is a multi-label sequence classification task.

We use a reduced batch size and sample size so that this can run on consumer hardware. We tested this on a Nvidia GTX 1080.

In [ ]:
import os

import torch
import pandas as pd
import numpy as np

from sklearn.metrics import recall_score, precision_score, f1_score, matthews_corrcoef, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

In [ ]:
DATA_PATH = './data/'
SEBERT_MODEL_PATH = './models/seBERT/'  # path of the pre-trained sebert model
SENTIMENT_MODEL_PATH = './models/sentiment/'  # path to store the final fine-tuned sentiment classification model
CHECKPOINTS_PATH = './models/checkpoints/'  # path to store checkpoints of the model for each epoch

In [ ]:
class Dataset(torch.utils.data.Dataset):
    """Just a standard torch Dataset for BERT-style data."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
def compute_metrics_multi_label(p):
    """This metrics computation is used by the huggingface trainer."""
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')
    mcc = matthews_corrcoef(y_true=labels, y_pred=pred)

    recall_ma = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision_ma = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1_ma = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {'accuracy': accuracy, 'precision_micro': precision, 'recall_micro': recall, 'f1_micro': f1, 'mcc': mcc, 'precision_macro': precision_ma, 'recall_macro': recall_ma, 'f1_macro': f1_ma}

In [ ]:
class seBERT(BaseEstimator, ClassifierMixin):
    """
    We are effectively wrapping the high-level Trainer and TrainingArguments classes from the Huggingface library into a
    scikit-learn classifier.
    This allows us to use all of scikit-learn in a more natural way, e.g., pipelines or grid search.
    """
    def __init__(self, checkpoints_dir='../checkpoints/', batch_size=8):
        self.trainer = None
        self.checkpoints_dir = checkpoints_dir
        self.model = BertForSequenceClassification.from_pretrained(SEBERT_MODEL_PATH + 'pytorch_model.bin', config=SEBERT_MODEL_PATH + 'config.json', num_labels=3)
        self.tokenizer = BertTokenizer.from_pretrained(SEBERT_MODEL_PATH, do_lower_case=True)
        self.batch_size = batch_size
        self.max_length = 128
            
    def fit(self, X, y):
        """fit implements simple fine-tuning from the pre-trained model.

        We split the training data into 80/20 training and validation sets, train for 5 epochs and chose the model
        that performs best on the validation data in the end.
        """
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
        X_train_tokens = self.tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=self.max_length)
        X_val_tokens = self.tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=self.max_length)

        train_dataset = Dataset(X_train_tokens, y_train)
        eval_dataset = Dataset(X_val_tokens, y_val)

        if not os.path.exists(self.checkpoints_dir):
            os.makedirs(self.checkpoints_dir)

        training_args = TrainingArguments(
            output_dir                  = self.checkpoints_dir,
            num_train_epochs            = 5,
            per_device_train_batch_size = self.batch_size,
            per_device_eval_batch_size  = self.batch_size,
            gradient_accumulation_steps = 4,
            eval_accumulation_steps     = 10,
            evaluation_strategy         = 'epoch',
            save_strategy               = 'epoch',
            load_best_model_at_end      = True
        )
        self.trainer = Trainer(
            model           = self.model,
            args            = training_args,
            train_dataset   = train_dataset,
            eval_dataset    = eval_dataset,
            compute_metrics = compute_metrics_multi_label
        )
        print(self.trainer.train())
        return self

    def predict_proba(self, X, y=None):
        """This is kept simple intentionally, for larger Datasets this would be too ineficient,
        because we would effectively force a batch size of 1."""
        y_probs = []
        self.trainer.model.eval()
        with torch.no_grad():
            for _, X_row in enumerate(X):
                inputs = self.tokenizer(X_row, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt").to('cuda')
                outputs = self.trainer.model(**inputs)
                probs = outputs[0].softmax(1).cpu().detach().numpy()
                y_probs.append(probs)
        return y_probs

    def predict(self, X, y=None):
        """Predict is evaluation."""
        y_probs = self.predict_proba(X, y)
        y_pred = []
        for y_prob in y_probs:
            y_pred.append(y_prob.argmax())
        return y_pred

    def save_model(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        self.trainer.model.save_pretrained(path)

In [ ]:
clf = seBERT(checkpoints_dir=CHECKPOINTS_PATH, batch_size=2)

# load a sample of the provided traianing data
df = pd.read_csv(DATA_PATH + 'github_gold.csv', sep=';').sample(n=1000)

# make labels numeric
def labelnum(row):
    if row['Polarity'] == 'negative':
        return 0
    elif row['Polarity'] == 'neutral':
        return 1
    elif row['Polarity'] == 'positive':
        return 2
    else:
        raise Exception('no such type!')

df['text_no_newlines'] = df['Text'].str.replace('\n', ' ')
df['label'] = df.apply(labelnum, axis=1)

X = df['text_no_newlines'].values
y = df['label'].astype(int).values

# fit all the data
clf.fit(X, y)

# save the fine-tuned model
clf.save_model(SENTIMENT_MODEL_PATH)